<a href="https://colab.research.google.com/github/AlexandreBourrieau/ML2/blob/main/RandomForest/RF_SML2010.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd

import random
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor

from keras import backend as K



---


# == Prédiction par RF ==


---



# Chargement et correction des données

Ce dataset est utilisé pour effectuer la prédiction de la température d'une pièce en fonction de plusieurs paramètres mesurés. La fréquence originale des données est d'une minute, puis a été modifiée à 15minutes avec un filtrage. L'ensemble correspond environ à une durée de 40 jours.  
Nous allons utiliser ici la température de la chambre comme cible et sélectionner 18 séries exogènes. 

**1. Chargement des données**

In [ ]:
!rm *.csv
!curl --location --remote-header-name --remote-name "https://github.com/AlexandreBourrieau/FICHIERS/raw/main/Series_Temporelles/Multi/Data/SMLselected_VSURF_pred.csv"


**2. Analyse et correction des données**

In [ ]:
# Création de la série sous Pandas
df_etude = pd.read_csv("SMLselected_VSURF_pred.csv")
df_etude

In [ ]:
df_etude = df_etude.drop(['Unnamed: 0'],axis=1)
df_etude

Affiche les types :

In [ ]:
df_etude.dtypes

Modifie les type en float32 :

In [ ]:
df_etude = df_etude.astype(dtype='float32')
df_etude.dtypes

# Séparation des données de test et d'entrainement

In [ ]:
# Sépare les données en entrainement et tests
pourcentage = 0.8
temps_separation = int(len(df_etude.values) * pourcentage)
date_separation = df_etude.index[temps_separation]

serie_entrainement_X = np.array(df_etude.values[:temps_separation],dtype=np.float32)
serie_test_X = np.array(df_etude.values[temps_separation:],dtype=np.float32)

print("Taille de l'entrainement : %d" %len(serie_entrainement_X))
print("Taille de la validation : %d" %len(serie_test_X))

**Normalisation des données :**

On normalise les données à l'aide de la fonction [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)

In [ ]:
from sklearn import preprocessing

# Constrution des séries
serie_entrainement_X_norm = []
serie_test_X_norm = []

for i in range(0,len(df_etude.columns)):
  serie_entrainement_X_norm.append(serie_entrainement_X[:,i])
  serie_test_X_norm.append(serie_test_X[:,i])

serie_entrainement_X_norm = tf.convert_to_tensor(serie_entrainement_X_norm)
serie_entrainement_X_norm = tf.transpose(serie_entrainement_X_norm)
serie_test_X_norm = tf.convert_to_tensor(serie_test_X_norm)
serie_test_X_norm = tf.transpose(serie_test_X_norm)

# Initialisaton du MinMaxScaler
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler.fit(serie_entrainement_X_norm)

# Normalisation des séries
serie_entrainement_X_norm = min_max_scaler.transform(serie_entrainement_X_norm)
serie_test_X_norm = min_max_scaler.transform(serie_test_X_norm)

In [ ]:
print(serie_entrainement_X_norm.shape)
print(serie_test_X_norm.shape)

In [ ]:
# Affiche quelques séries
fig, ax = plt.subplots(constrained_layout=True, figsize=(15,5))

ax.plot(df_etude.index[:temps_separation].values,serie_entrainement_X_norm[:,0:5], label="X_Ent")
ax.plot(df_etude.index[temps_separation:].values,serie_test_X_norm[:,0:5], label="X_Val")

ax.legend()
plt.show()

# Création du modèle RF

Recopie des X et des Y et décalage des Y :

In [ ]:
X_ent = serie_entrainement_X_norm[:-1,0:-1]
Y_ent = np.roll(serie_entrainement_X_norm[:,-1],shift=-1)
Y_ent = Y_ent[:-1]

In [ ]:
X_val = serie_test_X_norm[:-1,0:-1]

In [ ]:
# Informations sur les données
n = serie_entrainement_X_norm.shape[0]              # Nombre d'observations
p = serie_entrainement_X_norm.shape[1]              # Nombre de variables
mtry = np.int32(p/3)
n_arbres = 2000

clf = RandomForestRegressor(n_estimators=n_arbres, bootstrap=True, oob_score=True, max_features=mtry, n_jobs=-1)
clf.fit(X_ent,Y_ent)

# Prédictions par Random Forest

In [ ]:
pred_ent_forest = clf.predict(X_ent)
pred_val_forest = clf.predict(X_val)

In [ ]:
import plotly.graph_objects as go

decalage = 1

fig = go.Figure()

# Courbes originales
fig.add_trace(go.Scatter(x=df_etude.index,y=serie_entrainement_X_norm[:,-1],line=dict(color='blue', width=1)))
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation:],y=serie_test_X_norm[:,-1],line=dict(color='red', width=1)))

#Affiche les prédictions sur l'entrainement
fig.add_trace(go.Scatter(x=df_etude.index[1:temps_separation],y=pred_ent_forest, mode='lines', line=dict(color='green', width=1)))

#Affiche les prédictions sur les tests
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation+1:],y=pred_val_forest, mode='lines', line=dict(color='green', width=1)))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**Erreurs en single step**

In [ ]:
mse_ent = tf.keras.losses.mse(serie_entrainement_X_norm[1:,-1],pred_ent_forest)
mse_test = tf.keras.losses.mse(serie_test_X_norm[1:,-1],pred_val_forest)

In [ ]:
print(mse_ent)
print(mse_test)